In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential

import pathlib

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [2]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


2023-01-28 23:22:58.748238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-28 23:22:58.748401: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [3]:
data_dir = pathlib.Path('./Image data')
images=list(data_dir.glob('*/*.tif'))

In [4]:
image_dic= {
    'normal' : list(data_dir.glob("e0_Normal/*")),
    'hole':list(data_dir.glob("e1_hole/*")),
    'stain' : list(data_dir.glob("e2_Stain/*")),
    'net' :list(data_dir.glob("e3_Net/*")),
    'color' :list(data_dir.glob("e4_colour/*")),
    'crease': list(data_dir.glob("e5_Crease/*")),
}


In [5]:
labels = {
    'normal':0,
    'hole':1,
    'stain':2,
    'net':3,
    'color':4,
    'crease':5,
}

In [6]:
X,y=[],[]

for name, images in image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        
        
        if img is None:
            print('Wrong path:', image)
        else:
            resized = cv2.resize(img,IMG_SIZE)
            X.append(resized)
            y.append(labels[name])

Wrong path: Image data/e0_Normal/test1.tif.gz
Wrong path: Image data/e0_Normal/test.tif.gz
Wrong path: Image data/e0_Normal/test2.gif.gz


In [7]:
X = np.array(X)
y= np.array(y)

In [8]:

X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=0, shuffle=True)

In [16]:
X_train = X_train/255
X_test = X_test/255


In [17]:
print(len(X_train))
print(len(y_train))

903
903


In [18]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2

In [19]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    output_shape=(28,28,3),
    trainable=False)

OSError: SavedModel file does not exist at: /var/folders/_q/vp07tf1513jbt2ysl_y3k7k80000gn/T/tfhub_modules/145bb06ec3b59b08fb564ab752bd5aa222bfb50a/{saved_model.pbtxt|saved_model.pb}

In [20]:
classes = 6
# model = tf.keras.Sequential([feature_extractor_layer,
#             #tf.keras.layers.Conv2D(60,(3,3), activation = "relu", input_shape=(28,28,1)),
#             tf.keras.layers.Dropout(0.5, input_shape=(60,)),
#             tf.keras.layers.Dense(60, activation='sigmoid', name='predictions'),
#             tf.keras.layers.Dense(classes)])
# model.summary()
# model.compile(optimizer = "adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['acc'])
# model.fit(X_train,y_train,epochs=10)

In [16]:
model.evaluate(X_test,y_test)

2023-01-22 14:39:58.021872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 6s 523ms/step - loss: 1.0351 - acc: 0.6478


[1.035061001777649, 0.6478404998779297]

In [9]:
model1= tf.keras.applications.mobilenet.MobileNet()

In [10]:
model1.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [83]:
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=0, shuffle=True,test_size=0.3)
X_train = X_train/255
X_test = X_test/255

In [84]:
model_mn = tf.keras.Sequential()
for layes in model1.layers[:-22]:
    model_mn.add(layes)
model_mn.add(tf.keras.layers.GlobalAveragePooling2D())
model_mn.add(tf.keras.layers.Dense(60, activation='tanh'))
model_mn.add(tf.keras.layers.Dropout(0.6))
model_mn.add(tf.keras.layers.Dense(60,activation='tanh',name='predictions'))
model_mn.add(tf.keras.layers.Dense(6, activation='softmax'))



for layers in model_mn.layers[:-19]:
    layers.trainable= False


model_mn.compile(optimizer = Adam(learning_rate=0.0012), loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['acc'])
model_mn.fit(X_train,to_categorical(y_train),epochs=1)

2023-01-29 02:57:19.636197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 20s 579ms/step - loss: 0.2553 - acc: 0.9501


In [85]:
model_mn.evaluate(X_test,to_categorical(y_test))

2023-01-29 02:57:40.419852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 7s 463ms/step - loss: 0.1986 - acc: 0.9448


[0.19862812757492065, 0.9447514414787292]

In [87]:
predictions_arr =model_mn.predict(X_test)

predictions = []
for i in predictions_arr:
    predictions.append(np.argmax(i)) 

metric = tf.keras.metrics.Accuracy()
metric.update_state(y_test,predictions)
metric.result().numpy()

2023-01-29 18:23:38.926415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 6s 316ms/step


0.94475144

In [93]:
print(y_test[13],predictions[13])

3 3


In [43]:
a= model_mn.predict(np.array([X_test[0]]))
print(a)

2023-01-28 23:21:21.337109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
[[9.7095329e-01 4.1490371e-04 3.0123771e-03 3.3978738e-03 1.6745245e-02
  5.4762852e-03]]


In [367]:
model_mn.save("Model_super")

INFO:tensorflow:Assets written to: Model_super/assets


INFO:tensorflow:Assets written to: Model_super/assets


In [33]:
_model = keras.models.load_model("Model_super")

In [34]:
_model.evaluate(X_test,to_categorical(y_test))

2023-01-28 23:33:04.414686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 3s 208ms/step - loss: 0.1237 - acc: 0.9530


[0.1237167939543724, 0.9530386924743652]

In [35]:
import time
start = time.perf_counter()
_model.evaluate(X/255,to_categorical(y))
elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)


38/38 [==============================] - 7s 171ms/step - loss: 0.0670 - acc: 0.9784
Elapsed 8.326 seconds.


In [86]:
_model.summary()

Model: "sequential_142"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                          
                                                                 
 conv_dw_1_relu (ReLU)       (None, 112, 112, 32)   